# Data Compilation

In [4]:
# System modules
import os
import exdir as ex
import warnings
warnings.filterwarnings('always', category=UserWarning)
warnings.filterwarnings('always', category=DeprecationWarning)

# Data processing modules
import numpy as np
import nsync
from scipy.ndimage import gaussian_filter1d

# Visualization modules
import plotly.graph_objects as go

### Compile Data per FOV

In [5]:
root = "./data"
output_path = os.path.join("./", "compiled_data_2")
if not os.path.exists(output_path):
    os.mkdir(output_path)

f = ex.File(os.path.join(output_path, "data.exdir"), mode="w", allow_remove=True)

if os.path.isdir(root):
    for day in sorted(os.listdir(root)):
        day_path = os.path.join(root, day)
        if not os.path.isdir(day_path):
            continue

        # print(f"Processing: {day}")

        day_group_name = day.replace(" ", "_")
        day_group = f.create_group(day_group_name)
        day_group_dataset = []
        day_num_neurons = 0

        for animal in sorted(os.listdir(day_path)):
            animal_path = os.path.join(day_path, animal)
            if not os.path.isdir(animal_path):
                continue

            # print(f"  -> {animal}")

            animal_group_name = animal.replace(" ", "_")
            animal_group = day_group.create_group(animal_group_name)

            for fov in sorted(os.listdir(animal_path)):
                fov_path = os.path.join(animal_path, fov)
                if not os.path.isdir(fov_path):
                    continue

                # print("  " * 2 + f"  -> {fov}")

                fov_group_name = fov.replace(" ", "_")
                fov_group = animal_group.create_group(fov_group_name)

                extracted_signals_files = sorted([os.path.join(fov_path, f) for f in os.listdir(fov_path) if f.endswith(".npy") and "extractedsignals_raw" in f])
                behavior_event_log_files = sorted([os.path.join(fov_path, f) for f in os.listdir(fov_path) if f.endswith(".mat")])

                dataset = nsync.NSyncDataset(behavior_event_log_files, extracted_signals_files)
                day_num_neurons += dataset.num_neurons
                extracted_unique_events = dataset.extract_unique_events()
                extracted_windows = dataset.extract_normalized_windows()
                normalized_extracted_windows = nsync.normalize_windows(extracted_windows)
                if normalized_extracted_windows.ndim == 3:
                    day_group_dataset.append(normalized_extracted_windows)

        day_group_dataset = [np.nanmean(d, axis=0) for d in day_group_dataset]
        day_group_dataset = np.vstack(day_group_dataset)
        day_group.create_dataset("dataset", data=day_group_dataset)
        print(f"Day: {day}, Animals: {len(os.listdir(day_path))}, Num Neurons: {day_num_neurons}")

/home/thejoshbq/Desktop/PFC Self-Admin Analysis/nsync.py:57: RuntimeWarning:

divide by zero encountered in divide

/home/thejoshbq/Desktop/PFC Self-Admin Analysis/nsync.py:57: RuntimeWarning:

invalid value encountered in divide

/home/thejoshbq/Desktop/PFC Self-Admin Analysis/nsync.py:57: RuntimeWarning:

divide by zero encountered in divide

/home/thejoshbq/Desktop/PFC Self-Admin Analysis/nsync.py:57: RuntimeWarning:

invalid value encountered in divide



Day: 0 EarlyAcq, Animals: 17, Num Neurons: 2392
Day: 1 MidAcq, Animals: 17, Num Neurons: 2592


/home/thejoshbq/Desktop/PFC Self-Admin Analysis/nsync.py:57: RuntimeWarning:

divide by zero encountered in divide

/home/thejoshbq/Desktop/PFC Self-Admin Analysis/nsync.py:57: RuntimeWarning:

invalid value encountered in divide



Day: 2 LateAcq, Animals: 17, Num Neurons: 2237
Insufficient target events found.
Insufficient target events found.


/home/thejoshbq/Desktop/PFC Self-Admin Analysis/nsync.py:57: RuntimeWarning:

divide by zero encountered in divide

/home/thejoshbq/Desktop/PFC Self-Admin Analysis/nsync.py:57: RuntimeWarning:

invalid value encountered in divide



Insufficient target events found.
Insufficient target events found.
Day: 3 EarlyExt, Animals: 17, Num Neurons: 2284
Insufficient target events found.
Insufficient target events found.
Insufficient target events found.
Insufficient target events found.
Insufficient target events found.
Insufficient target events found.
Insufficient target events found.
Insufficient target events found.
Insufficient target events found.
Insufficient target events found.
Insufficient target events found.
Insufficient target events found.
Insufficient target events found.
Insufficient target events found.
Day: 4 LastExt, Animals: 17, Num Neurons: 2198
Insufficient target events found.
Day: 5 CueRein, Animals: 17, Num Neurons: 2192
Insufficient target events found.
Day: 6 DrugRein, Animals: 16, Num Neurons: 2381
Day: 7 TMTRein, Animals: 16, Num Neurons: 2223


In [6]:
infile = ex.File("./compiled_data_2/data", mode="r")
# dataset = infile["1_MidAcq"]["dataset"]["event_window_extracted_signals"]

fig = go.Figure()

for day in list(infile.keys())[:3]:
    dataset = None
    try:
        if "dataset" in list(infile[day].keys()):
            dataset = infile[day]["dataset"]
        else:
            print(f"Day: {day}, No Dataset")
            continue
    except Exception as e:
        print(f"Error loading {day}: {e}")

    if dataset:
        mean_signal = gaussian_filter1d(np.nanmean(dataset, axis=0), sigma=1)
        fig.add_trace(
            go.Scatter(
                y=mean_signal,
                mode='lines',
                name=f"{day}, {dataset.shape[0]} neurons"
            )
        )

        # median_signal = np.median(dataset, axis=0)
        # fig.add_trace(
        #     go.Scatter(
        #         y=median_signal,
        #         mode='lines',
        #         name=day
        #     )
        # )

fig.update_layout(
    title='Signal Over Time',
    xaxis_title='Sample Index',
    yaxis_title='Signal Value',
    legend_title="Dataset",
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    ),
    width=800,
    height=600
)

fig.show()
